In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 161
USE = 'clicks'

PART = 3

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(f"../../data/candidate_scores/submission_v711_LB.csv")
df = df.loc[df.session_type.str.contains(USE)]
df = cudf.DataFrame(df)
print( df.shape )
df.head()

(1671803, 2)


,session_type,labels
0,12899779_clicks,59625 737445 1253524 731692 1246235 302827 286...
1,12899780_clicks,1142000 736515 973453 582732 1502122 889686 48...
2,12899781_clicks,918667 199008 194067 57315 141736 1681537 1836...
3,12899782_clicks,834354 595994 740494 889671 987399 779477 1344...
4,12899783_clicks,1817895 607638 1754419 1216820 1729553 300127 ...


In [4]:
df2 = pd.read_csv(f"../../data/candidate_scores/submission_v711_wgt_LB.csv")
df2 = df2.loc[df2.session_type.str.contains(USE)]
df2 = cudf.DataFrame(df2)
print( df2.shape )
df2.head()

(1671803, 2)


,session_type,labels
0,12899779_clicks,-1 10.495000000000001 6.0 5.99 5.9350000000000...
1,12899780_clicks,-1 -1 -1 -1 21.776739999999997 20.81098 17.662...
2,12899781_clicks,-1 -1 -1 -1 -1 13.5 10.985 10.98 10.92 10.4950...
3,12899782_clicks,12.671315804076116 6.191834812668787 5.8893487...
4,12899783_clicks,-1 -1 -1 -1 -1 -1 -1 -1 -1 8.93 8.5 7.465 6.99...


In [5]:
# FILTER USERS
UU = np.load(f'/raid/Kaggle/otto/data0/test_user_{PART}.npy')
print( len(UU) )

417950


In [6]:
df['user'] = df.session_type.str.partition('_')[0].astype('int32')
df = df.loc[df.user.isin(UU)]
df = df.drop('session_type',axis=1)
df.head()

,labels,user
0,59625 737445 1253524 731692 1246235 302827 286...,12899779
1,1142000 736515 973453 582732 1502122 889686 48...,12899780
2,918667 199008 194067 57315 141736 1681537 1836...,12899781
3,834354 595994 740494 889671 987399 779477 1344...,12899782
4,1817895 607638 1754419 1216820 1729553 300127 ...,12899783


In [7]:
df2['user'] = df2.session_type.str.partition('_')[0].astype('int32')
df2 = df2.loc[df2.user.isin(UU)]
df2 = df2.drop('session_type',axis=1)
df2.head()

,labels,user
0,-1 10.495000000000001 6.0 5.99 5.9350000000000...,12899779
1,-1 -1 -1 -1 21.776739999999997 20.81098 17.662...,12899780
2,-1 -1 -1 -1 -1 13.5 10.985 10.98 10.92 10.4950...,12899781
3,12.671315804076116 6.191834812668787 5.8893487...,12899782
4,-1 -1 -1 -1 -1 -1 -1 -1 -1 8.93 8.5 7.465 6.99...,12899783


In [8]:
tmp = df.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,item,pos
0,59625,0
0,737445,1
0,1253524,2
0,731692,3
0,1246235,4


In [9]:
users = df[['user']].copy()
users.head()

,user
0,12899779
1,12899780
2,12899781
3,12899782
4,12899783


In [10]:
users = users.merge(tmp, left_index=True, right_index=True)

In [11]:
del df,tmp
gc.collect()

58

In [12]:
users.head()

,user,item,pos
28,12899807,1830634,80
28,12899807,1803229,81
28,12899807,1048057,82
28,12899807,1403260,83
28,12899807,1294654,84


In [13]:
users.dtypes

user    int32
item    int32
pos     int32
dtype: object

In [14]:
tmp = df2.labels.str.split(' ').explode().astype('float32').rename('wgt').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,wgt,pos
0,-1.000,0
0,10.495,1
0,6.000,2
0,5.990,3
0,5.935,4


In [15]:
users2 = df2[['user']].copy()#.reset_index()
users2.head()

,user
0,12899779
1,12899780
2,12899781
3,12899782
4,12899783


In [16]:
users2 = users2.merge(tmp, left_index=True, right_index=True)

In [17]:
users2.head()

,user,wgt,pos
14,12899793,2.50000,88
14,12899793,2.42000,89
14,12899793,2.35500,90
14,12899793,2.10775,91
14,12899793,2.09850,92


In [18]:
users = users.merge(users2,on=['user','pos'],how='left')

In [19]:
users.head()

,user,item,pos,wgt
0,12899974,1762295,36,4.913460
1,12899974,1456742,37,4.880000
2,12899974,1319266,38,4.637995
3,12899974,382573,39,4.575250
4,12899974,1428431,40,4.395000


In [20]:
del users2
gc.collect()

62

In [21]:
users.shape

(41795000, 4)

# Add Weights

In [22]:
for k in [1,3,4,5,6,7]:

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v11_v{k}.csv')
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v11_wgt_v{k}.csv')
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{k}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{k}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

1 (41795000, 5)
3 (41795000, 6)
4 (41795000, 7)
5 (41795000, 8)
6 (41795000, 9)
7 (41795000, 10)


In [23]:
#import gc
#users = users.to_pandas()
#gc.collect()

In [24]:
USE2 = 'orders'
OFFSET = 10
for j,k in enumerate(['../../data/candidate_scores/submission_v11_B',
                      '../../data/candidate_scores/submission_v13']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-6c-22-otto/submission_v11_B (41795000, 11)
../../../Dec-2022/dec-18-22-otto/submission_v13 (41795000, 12)


In [25]:
USE2 = 'clicks'
OFFSET = 12
for j,k in enumerate(['../../data/candidate_scores/submission_v11_B',
                      '../../data/candidate_scores/submission_v13']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-6c-22-otto/submission_v11_B (41795000, 13)
../../../Dec-2022/dec-18-22-otto/submission_v13 (41795000, 14)


In [26]:
#users = users.to_pandas()
#gc.collect()

In [27]:
if USE=='clicks': USE2 = 'orders'
else: USE2 = 'clicks'
    
OFFSET = 30
for j,k in enumerate([f"../../data/candidate_scores/submission_v711_LB"]):

    dfA = pd.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.loc[dfA.user.isin(UU)]
    dfA = dfA.drop('session_type',axis=1)
    dfA = cudf.DataFrame(dfA)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(100),tmpA.shape[0]//100 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = pd.read_csv(f'../../data/candidate_scores/submission_v711_wgt_LB.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.loc[dfB.user.isin(UU)]
    dfB = dfB.drop('session_type',axis=1)
    dfB = cudf.DataFrame(dfB)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(100),tmpB.shape[0]//100 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

/raid/Kaggle/otto/data0/submission_v711_LB (41795000, 15)


In [28]:
USE2 = 'orders'
OFFSET = 14

for j,k in enumerate( range(20,28) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v11_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v11_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

20 (41795000, 16)
21 (41795000, 17)
22 (41795000, 18)
23 (41795000, 19)
24 (41795000, 20)
25 (41795000, 21)
26 (41795000, 22)
27 (41795000, 23)


In [29]:
USE2 = 'orders'
OFFSET = 22
for j,k in enumerate(['../../data/candidate_scores/submission_v14']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-18-22-otto/submission_v14 (41795000, 24)


In [30]:
USE2 = 'clicks'
OFFSET = 23
for j,k in enumerate(['../../data/candidate_scores/submission_v14']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-18-22-otto/submission_v14 (41795000, 25)


In [31]:
#USE2 = 'orders'
OFFSET = 24

for j,(k,USE2) in enumerate( zip( range(31,34),['orders','orders','clicks'] ) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v11_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v11_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

31 (41795000, 26)
32 (41795000, 27)
33 (41795000, 28)


In [32]:
users = users.to_pandas()
gc.collect()

0

In [33]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26
0,12900204,814820,85,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0
1,12900204,1414685,87,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0
2,12900204,655188,79,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0
3,12900204,927099,80,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0
4,12900204,804249,84,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0


# User and Item Features

In [34]:
item_features = pd.read_parquet('../../data/item_user_features/item14b_LB.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

(1848283, 7)


,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
aid,,,,,,,
0,0.0,20.0,17.0,1.176471,0.0,<NA>,<NA>
1,0.052632,19.0,17.0,1.117647,0.229416,<NA>,<NA>
2,0.0,11.0,10.0,1.1,0.0,<NA>,<NA>
3,0.104082,1470.0,715.0,2.055944,0.353024,<NA>,<NA>
4,0.0,64.0,51.0,1.254902,0.0,<NA>,<NA>


In [35]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
0,12900204,814820,85,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.295082,61.0,34.0,1.794118,0.5581,<NA>,<NA>
1,12900204,1414685,87,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.030695,619.0,450.0,1.375556,0.190455,<NA>,<NA>
2,12900204,655188,79,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.026008,769.0,566.0,1.358657,0.167238,<NA>,<NA>
3,12900204,927099,80,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.035363,509.0,362.0,1.406077,0.205071,<NA>,<NA>
4,12900204,804249,84,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.126332,657.0,446.0,1.473094,0.387524,<NA>,<NA>


In [36]:
item_features = pd.read_parquet('../../data/item_user_features/item13_LB.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

count_item_0_6 int32
count_item_1_6 int32
count_item_2_6 int32
count_item_3_6 int32
count_item_4_6 int32
count_item_5_6 int32
count_item_6_6 int32
count_user_0_6 int32
count_user_1_6 int32
count_user_2_6 int32
count_user_3_6 int32
count_user_4_6 int32
count_user_5_6 int32
count_user_6_6 int32
(783486, 23)


,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6
5576,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2.0,0.755929,0.755929
5579,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964
5580,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.786796,0.786796
5582,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.377964,0.377964
5584,-1.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,0.487950,0.487950


In [37]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6
0,12900204,814820,85,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.295082,61.0,34.0,1.794118,0.5581,<NA>,<NA>,0.0,-1.000000,-1.000000,-1.000000,0.000000,-1.0,0.000000,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,0.534522,0.534522
1,12900204,1414685,87,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.030695,619.0,450.0,1.375556,0.190455,<NA>,<NA>,-1.0,-1.000000,-1.000000,0.000000,-1.000000,0.0,-1.000000,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,0.487950,0.487950
2,12900204,655188,79,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.026008,769.0,566.0,1.358657,0.167238,<NA>,<NA>,-1.0,-1.000000,-1.000000,0.000000,0.000000,0.0,0.045455,-1.0,-1.0,-1.0,5.0,9.0,12.0,22.0,-1.0,-1.0,-1.0,5.0,6.0,10.0,15.0,8.214389,8.214389
3,12900204,927099,80,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.035363,509.0,362.0,1.406077,0.205071,<NA>,<NA>,0.0,0.000000,0.142857,0.066667,0.000000,0.1,0.000000,5.0,4.0,7.0,15.0,11.0,10.0,13.0,4.0,4.0,6.0,12.0,7.0,8.0,9.0,4.111540,4.111540
4,12900204,804249,84,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.126332,657.0,446.0,1.473094,0.387524,<NA>,<NA>,0.0,0.111111,0.142857,0.000000,0.315789,0.0,0.000000,4.0,9.0,7.0,6.0,19.0,4.0,10.0,3.0,5.0,5.0,4.0,7.0,3.0,6.0,5.191568,5.191568


In [38]:
item_features = pd.read_parquet('../../data/item_user_features/item4b_LB.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

count_item int32
count_user int32
(1839558, 9)


,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
0,0.000000,33.0,29.0,50843.453125,388588.906250,1.137931,0.000000,21374.218750,148150.937500
1,0.032258,31.0,27.0,49398.097656,258430.359375,1.148148,0.179605,18655.564453,160308.437500
2,0.000000,16.0,15.0,44175.187500,292575.187500,1.066667,0.000000,10662.440430,133456.968750
3,0.103150,2540.0,1268.0,50379.726562,293184.125000,2.003155,0.349970,16550.375000,148045.750000
4,0.049080,163.0,103.0,54755.527344,299113.812500,1.582524,0.216700,18486.285156,161496.953125


In [39]:
#users = users.to_pandas()

In [40]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')

In [41]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
0,12900204,814820,85,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.295082,61.0,34.0,1.794118,0.5581,<NA>,<NA>,0.0,-1.000000,-1.000000,-1.000000,0.000000,-1.0,0.000000,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,0.534522,0.534522,0.235294,102.0,63.0,48353.234375,300776.75000,1.619048,0.510804,20002.632812,176633.484375
1,12900204,1414685,87,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.030695,619.0,450.0,1.375556,0.190455,<NA>,<NA>,-1.0,-1.000000,-1.000000,0.000000,-1.000000,0.0,-1.000000,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,0.487950,0.487950,0.052270,727.0,478.0,50428.050781,337081.15625,1.520921,0.267665,19778.921875,165945.156250
2,12900204,655188,79,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.026008,769.0,566.0,1.358657,0.167238,<NA>,<NA>,-1.0,-1.000000,-1.000000,0.000000,0.000000,0.0,0.045455,-1.0,-1.0,-1.0,5.0,9.0,12.0,22.0,-1.0,-1.0,-1.0,5.0,6.0,10.0,15.0,8.214389,8.214389,0.024841,1570.0,1137.0,53590.980469,325448.31250,1.380827,0.163672,19221.863281,175220.234375
3,12900204,927099,80,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.035363,509.0,362.0,1.406077,0.205071,<NA>,<NA>,0.0,0.000000,0.142857,0.066667,0.000000,0.1,0.000000,5.0,4.0,7.0,15.0,11.0,10.0,13.0,4.0,4.0,6.0,12.0,7.0,8.0,9.0,4.111540,4.111540,0.043238,1087.0,723.0,50464.691406,311095.40625,1.503458,0.220847,19701.205078,172401.671875
4,12900204,804249,84,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.126332,657.0,446.0,1.473094,0.387524,<NA>,<NA>,0.0,0.111111,0.142857,0.000000,0.315789,0.0,0.000000,4.0,9.0,7.0,6.0,19.0,4.0,10.0,3.0,5.0,5.0,4.0,7.0,3.0,6.0,5.191568,5.191568,0.119103,1293.0,858.0,54065.148438,300702.75000,1.506993,0.377028,20131.054688,168841.062500


In [42]:
del item_features
gc.collect()

0

In [43]:
item_features = pd.read_parquet('../../data/item_user_features/item10_LB.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
users = users.merge(item_features, left_on='item', right_index=True, how='left')
users.head()

count_item3 int32
count_user3 int32
prev3 int32
next3 int32
orders3 int32
carts3 int32
(783486, 15)


,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
0,12900204,814820,85,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.295082,61.0,34.0,1.794118,0.5581,<NA>,<NA>,0.0,-1.000000,-1.000000,-1.000000,0.000000,-1.0,0.000000,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,0.534522,0.534522,0.235294,102.0,63.0,48353.234375,300776.75000,1.619048,0.510804,20002.632812,176633.484375,0.000000,3.0,3.0,48235.332031,278635.343750,1.000000,0.000000,12842.814453,120316.984375,3.0,1.0,0.0,0.0,-1.0,-1.0
1,12900204,1414685,87,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.030695,619.0,450.0,1.375556,0.190455,<NA>,<NA>,-1.0,-1.000000,-1.000000,0.000000,-1.000000,0.0,-1.000000,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,0.487950,0.487950,0.052270,727.0,478.0,50428.050781,337081.15625,1.520921,0.267665,19778.921875,165945.156250,0.000000,2.0,2.0,47927.500000,134327.500000,1.000000,0.000000,25340.585938,96847.468750,1.0,2.0,0.0,0.0,-1.0,-1.0
2,12900204,655188,79,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.026008,769.0,566.0,1.358657,0.167238,<NA>,<NA>,-1.0,-1.000000,-1.000000,0.000000,0.000000,0.0,0.045455,-1.0,-1.0,-1.0,5.0,9.0,12.0,22.0,-1.0,-1.0,-1.0,5.0,6.0,10.0,15.0,8.214389,8.214389,0.024841,1570.0,1137.0,53590.980469,325448.31250,1.380827,0.163672,19221.863281,175220.234375,0.020833,48.0,36.0,54928.019531,233128.015625,1.333333,0.144338,18985.285156,89476.882812,25.0,25.0,0.0,1.0,-1.0,1.0
3,12900204,927099,80,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.035363,509.0,362.0,1.406077,0.205071,<NA>,<NA>,0.0,0.000000,0.142857,0.066667,0.000000,0.1,0.000000,5.0,4.0,7.0,15.0,11.0,10.0,13.0,4.0,4.0,6.0,12.0,7.0,8.0,9.0,4.111540,4.111540,0.043238,1087.0,723.0,50464.691406,311095.40625,1.503458,0.220847,19701.205078,172401.671875,0.046154,65.0,48.0,55887.167969,253942.546875,1.354167,0.211451,17338.414062,166319.750000,45.0,41.0,0.0,3.0,-1.0,1.0
4,12900204,804249,84,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.126332,657.0,446.0,1.473094,0.387524,<NA>,<NA>,0.0,0.111111,0.142857,0.000000,0.315789,0.0,0.000000,4.0,9.0,7.0,6.0,19.0,4.0,10.0,3.0,5.0,5.0,4.0,7.0,3.0,6.0,5.191568,5.191568,0.119103,1293.0,858.0,54065.148438,300702.75000,1.506993,0.377028,20131.054688,168841.062500,0.135593,59.0,32.0,59197.136719,280322.562500,1.843750,0.392060,19012.048828,157358.687500,33.0,34.0,1.0,6.0,1.0,1.0


In [44]:
del item_features
gc.collect()

0

In [45]:
user_features = pd.read_parquet('../../data/item_user_features/user4_LB.pqt')
for c in user_features.columns:
    if str(user_features[c].dtype)=='int32':
        print(c, user_features[c].dtype)
        user_features[c] = user_features[c].astype('float32')
print( user_features.shape )
user_features.head()

count_item2 int32
unique_item2 int32
(1671803, 9)


,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
session,,,,,,,,,
12899779,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
12899780,0.000000,5.0,4.0,79291.601562,338491.59375,1.250000,0.000000,62.875275,62.875275
12899781,0.090909,11.0,5.0,68086.093750,288013.37500,2.200000,0.301511,12359.510742,143144.843750
12899782,0.457143,70.0,38.0,64777.812500,398034.96875,1.842105,0.695445,14570.271484,27593.218750
12899783,0.000000,11.0,9.0,70573.726562,329773.71875,1.222222,0.000000,17300.041016,95597.734375


In [46]:
users = users.merge(user_features, left_on='user', right_index=True, how='left')

In [47]:
del user_features
gc.collect()

0

In [48]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
0,12900204,814820,85,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.295082,61.0,34.0,1.794118,0.5581,<NA>,<NA>,0.0,-1.000000,-1.000000,-1.000000,0.000000,-1.0,0.000000,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,0.534522,0.534522,0.235294,102.0,63.0,48353.234375,300776.75000,1.619048,0.510804,20002.632812,176633.484375,0.000000,3.0,3.0,48235.332031,278635.343750,1.000000,0.000000,12842.814453,120316.984375,3.0,1.0,0.0,0.0,-1.0,-1.0,0.0,3.0,2.0,64868.667969,439268.65625,1.5,0.0,12538.925781,87237.476562
1,12900204,1414685,87,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.030695,619.0,450.0,1.375556,0.190455,<NA>,<NA>,-1.0,-1.000000,-1.000000,0.000000,-1.000000,0.0,-1.000000,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,0.487950,0.487950,0.052270,727.0,478.0,50428.050781,337081.15625,1.520921,0.267665,19778.921875,165945.156250,0.000000,2.0,2.0,47927.500000,134327.500000,1.000000,0.000000,25340.585938,96847.468750,1.0,2.0,0.0,0.0,-1.0,-1.0,0.0,3.0,2.0,64868.667969,439268.65625,1.5,0.0,12538.925781,87237.476562
2,12900204,655188,79,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.026008,769.0,566.0,1.358657,0.167238,<NA>,<NA>,-1.0,-1.000000,-1.000000,0.000000,0.000000,0.0,0.045455,-1.0,-1.0,-1.0,5.0,9.0,12.0,22.0,-1.0,-1.0,-1.0,5.0,6.0,10.0,15.0,8.214389,8.214389,0.024841,1570.0,1137.0,53590.980469,325448.31250,1.380827,0.163672,19221.863281,175220.234375,0.020833,48.0,36.0,54928.019531,233128.015625,1.333333,0.144338,18985.285156,89476.882812,25.0,25.0,0.0,1.0,-1.0,1.0,0.0,3.0,2.0,64868.667969,439268.65625,1.5,0.0,12538.925781,87237.476562
3,12900204,927099,80,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.035363,509.0,362.0,1.406077,0.205071,<NA>,<NA>,0.0,0.000000,0.142857,0.066667,0.000000,0.1,0.000000,5.0,4.0,7.0,15.0,11.0,10.0,13.0,4.0,4.0,6.0,12.0,7.0,8.0,9.0,4.111540,4.111540,0.043238,1087.0,723.0,50464.691406,311095.40625,1.503458,0.220847,19701.205078,172401.671875,0.046154,65.0,48.0,55887.167969,253942.546875,1.354167,0.211451,17338.414062,166319.750000,45.0,41.0,0.0,3.0,-1.0,1.0,0.0,3.0,2.0,64868.667969,439268.65625,1.5,0.0,12538.925781,87237.476562
4,12900204,804249,84,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.126332,657.0,446.0,1.473094,0.387524,<NA>,<NA>,0.0,0.111111,0.142857,0.000000,0.315789,0.0,0.000000,4.0,9.0,7.0,6.0,19.0,4.0,10.0,3.0,5.0,5.0,4.0,7.0,3.0,6.0,5.191568,5.191568,0.119103,1293.0,858.0,54065.148438,300702.75000,1.506993,0.377028,20131.054688,168841.062500,0.135593,59.0,32.0,59197.136719,280322.562500,1.843750,0.392060,19012.048828,157358.687500,33.0,34.0,1.0,6.0,1.0,1.0,0.0,3.0,2.0,64868.6

In [49]:
for c in users.columns:
    print(c,users[c].dtype)

user int32
item int32
pos int32
wgt float32
score_1 float32
score_3 float32
score_4 float32
score_5 float32
score_6 float32
score_7 float32
score_10 float32
score_11 float32
score_12 float32
score_13 float32
score_30 float32
score_14 float32
score_15 float32
score_16 float32
score_17 float32
score_18 float32
score_19 float32
score_20 float32
score_21 float32
score_22 float32
score_23 float32
score_24 float32
score_25 float32
score_26 float32
buy_ratio_5 Float32
count_item_5 Float32
count_user_5 Float32
repeat_5 Float32
buy_ratio_std_5 Float32
item_day_std_5 Float32
user_day_std_5 Float32
buy_ratio_0_6 float32
buy_ratio_1_6 float32
buy_ratio_2_6 float32
buy_ratio_3_6 float32
buy_ratio_4_6 float32
buy_ratio_5_6 float32
buy_ratio_6_6 float32
count_item_0_6 float32
count_item_1_6 float32
count_item_2_6 float32
count_item_3_6 float32
count_item_4_6 float32
count_item_5_6 float32
count_item_6_6 float32
count_user_0_6 float32
count_user_1_6 float32
count_user_2_6 float32
count_user_3_6 float3

In [50]:
for c in users.columns:
    t = str(users[c].dtype)
    if (t=='int64'): 
        users[c] = users[c].astype('int32')
        print('convert',c,'to int32')
    if (t=='float64')|(t=='Int32')|(t=='Float32'):
        users[c] = users[c].astype('float32')
        print('convert',c,'to float32')

convert buy_ratio_5 to float32
convert count_item_5 to float32
convert count_user_5 to float32
convert repeat_5 to float32
convert buy_ratio_std_5 to float32
convert item_day_std_5 to float32
convert user_day_std_5 to float32


In [51]:
#%%time
#users = users.sort_values(['user','pos']).reset_index(drop=True)

In [52]:
#users = users.reset_index(drop=True)
#users = users.to_pandas()
#gc.collect()

In [53]:
users.head()

,user,item,pos,wgt,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
0,12900204,814820,85,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.295082,61.0,34.0,1.794118,0.558100,NaN,NaN,0.0,-1.000000,-1.000000,-1.000000,0.000000,-1.0,0.000000,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,0.534522,0.534522,0.235294,102.0,63.0,48353.234375,300776.75000,1.619048,0.510804,20002.632812,176633.484375,0.000000,3.0,3.0,48235.332031,278635.343750,1.000000,0.000000,12842.814453,120316.984375,3.0,1.0,0.0,0.0,-1.0,-1.0,0.0,3.0,2.0,64868.667969,439268.65625,1.5,0.0,12538.925781,87237.476562
1,12900204,1414685,87,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.030695,619.0,450.0,1.375556,0.190455,NaN,NaN,-1.0,-1.000000,-1.000000,0.000000,-1.000000,0.0,-1.000000,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,0.487950,0.487950,0.052270,727.0,478.0,50428.050781,337081.15625,1.520921,0.267665,19778.921875,165945.156250,0.000000,2.0,2.0,47927.500000,134327.500000,1.000000,0.000000,25340.585938,96847.468750,1.0,2.0,0.0,0.0,-1.0,-1.0,0.0,3.0,2.0,64868.667969,439268.65625,1.5,0.0,12538.925781,87237.476562
2,12900204,655188,79,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.026008,769.0,566.0,1.358657,0.167238,NaN,NaN,-1.0,-1.000000,-1.000000,0.000000,0.000000,0.0,0.045455,-1.0,-1.0,-1.0,5.0,9.0,12.0,22.0,-1.0,-1.0,-1.0,5.0,6.0,10.0,15.0,8.214389,8.214389,0.024841,1570.0,1137.0,53590.980469,325448.31250,1.380827,0.163672,19221.863281,175220.234375,0.020833,48.0,36.0,54928.019531,233128.015625,1.333333,0.144338,18985.285156,89476.882812,25.0,25.0,0.0,1.0,-1.0,1.0,0.0,3.0,2.0,64868.667969,439268.65625,1.5,0.0,12538.925781,87237.476562
3,12900204,927099,80,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.035363,509.0,362.0,1.406077,0.205071,NaN,NaN,0.0,0.000000,0.142857,0.066667,0.000000,0.1,0.000000,5.0,4.0,7.0,15.0,11.0,10.0,13.0,4.0,4.0,6.0,12.0,7.0,8.0,9.0,4.111540,4.111540,0.043238,1087.0,723.0,50464.691406,311095.40625,1.503458,0.220847,19701.205078,172401.671875,0.046154,65.0,48.0,55887.167969,253942.546875,1.354167,0.211451,17338.414062,166319.750000,45.0,41.0,0.0,3.0,-1.0,1.0,0.0,3.0,2.0,64868.667969,439268.65625,1.5,0.0,12538.925781,87237.476562
4,12900204,804249,84,1.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,0.126332,657.0,446.0,1.473094,0.387524,NaN,NaN,0.0,0.111111,0.142857,0.000000,0.315789,0.0,0.000000,4.0,9.0,7.0,6.0,19.0,4.0,10.0,3.0,5.0,5.0,4.0,7.0,3.0,6.0,5.191568,5.191568,0.119103,1293.0,858.0,54065.148438,300702.75000,1.506993,0.377028,20131.054688,168841.062500,0.135593,59.0,32.0,59197.136719,280322.562500,1.843750,0.392060,19012.048828,157358.687500,33.0,34.0,1.0,6.0,1.0,1.0,0.0,3.0,2.0,64868.667969,43

In [54]:
users.to_parquet(f'../../data/infer_with_features/train_v{VER}_LB_{PART}.pqt',index=False)